## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import the API key.
import sys, os
sys.path.append(os.path.abspath(os.path.join('..')))

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,College,US,64.8569,-147.8028,56.80,74,75,3.44,broken clouds
1,1,San Andres,CO,12.5847,-81.7006,86.02,74,40,12.66,scattered clouds
2,2,Iracoubo,GF,5.4802,-53.2011,82.85,72,39,8.57,scattered clouds
3,3,Mount Gambier,AU,-37.8333,140.7667,47.23,85,62,8.05,broken clouds
4,4,Harper,LR,4.3750,-7.7169,77.50,66,66,8.68,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  50
What is the maximum temperature you would like for your trip?  80


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,College,US,64.8569,-147.8028,56.80,74,75,3.44,broken clouds
4,4,Harper,LR,4.3750,-7.7169,77.50,66,66,8.68,broken clouds
9,9,Vaini,TO,-21.2000,-175.2000,66.36,100,20,4.61,few clouds
10,10,Grindavik,IS,63.8424,-22.4338,55.35,94,75,9.22,light intensity drizzle
11,11,Caravelas,BR,-17.7125,-39.2481,77.49,63,2,8.46,clear sky
12,12,Rikitea,PF,-23.1203,-134.9692,74.55,73,36,12.15,scattered clouds
13,13,Khatanga,RU,71.9667,102.5000,52.68,64,100,7.54,overcast clouds
16,16,Tuktoyaktuk,CA,69.4541,-133.0374,51.80,87,100,2.30,overcast clouds
18,18,Kodiak,US,57.7900,-152.4072,51.31,93,100,3.44,overcast clouds
20,20,Airai,TL,-8.9266,125.4092,55.44,90,88,2.80,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                5
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_no_nulls_df = preferred_cities_df.dropna() 

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_no_nulls_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,College,US,56.80,broken clouds,64.8569,-147.8028,
4,Harper,LR,77.50,broken clouds,4.3750,-7.7169,
9,Vaini,TO,66.36,few clouds,-21.2000,-175.2000,
10,Grindavik,IS,55.35,light intensity drizzle,63.8424,-22.4338,
11,Caravelas,BR,77.49,clear sky,-17.7125,-39.2481,
12,Rikitea,PF,74.55,scattered clouds,-23.1203,-134.9692,
13,Khatanga,RU,52.68,overcast clouds,71.9667,102.5000,
16,Tuktoyaktuk,CA,51.80,overcast clouds,69.4541,-133.0374,
18,Kodiak,US,51.31,overcast clouds,57.7900,-152.4072,
20,Airai,TL,55.44,overcast clouds,-8.9266,125.4092,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    try:
    
        # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping ")
        
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found...skipping 
Hotel not found.

In [10]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,College,US,56.80,broken clouds,64.8569,-147.8028,Pike's Waterfront Lodge
4,Harper,LR,77.50,broken clouds,4.3750,-7.7169,Screensaver
9,Vaini,TO,66.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
10,Grindavik,IS,55.35,light intensity drizzle,63.8424,-22.4338,Northern Light Inn & Max's Restaurant
11,Caravelas,BR,77.49,clear sky,-17.7125,-39.2481,Pousada dos Navegantes
...,...,...,...,...,...,...,...
710,Penha,BR,70.48,scattered clouds,-26.7694,-48.6458,Hotel Mirante do Parque
711,Ust-Barguzin,RU,55.44,overcast clouds,53.4500,108.9833,Barguzinskiy Priboy Inn
712,Igrim,RU,62.65,broken clouds,63.1906,64.4162,Obshchezhitiye Uchebno-Proizvodstvennogo Tsentra
713,Ust-Ilimsk,RU,57.74,broken clouds,58.0006,102.6619,AleNat Center


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(50.0, -20.0), zoom_level=3)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# # 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))